[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/alesaccoia/IULM_DDM2324_Notebooks/blob/main/06_calcolo_bdi_cdi.ipynb)

# Esercizio calcolo BDI/CDI (sezione 3.2.2 del libro)

Si comincia importando il dataset

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_excel('data/calcolo_bdi_cdi.xlsx')
print(df)

Vediamo per prima cosa la dimensione del dataset

In [ ]:
totale_campione = len(df)
totale_campione

Stampiamo a schermo i brand unici all'interno del dataset

In [ ]:
target_brands = df['Brand'].unique()
target_brands

Ripuliamo la lista dal brand "nan", valore che indica che il consumatore non acquista nessun brand.

Utilizza il caratteristico costrutto chiamato [list comprehension](https://www.w3schools.com/python/python_lists_comprehension.asp)

In [ ]:
target_brands = [b for b in target_brands if b is not np.nan]
target_brands

Calcoliamo il totale di rispondenti che acquista un brand della categoria merceologica, di qualsiasi brand

In [ ]:
totale_consumatori_categoria = len(df[df['Brand'].notnull()])
totale_consumatori_categoria

La penetrazione della categoria sulla popolazione é il rapporto tra il numero di consumatori che consuma e il totale di consumatori nel campione

In [ ]:
penetrazione_categoria = totale_consumatori_categoria / totale_campione
penetrazione_categoria

Possiamo calcolare le proporzioni di persone del campione che acquistano ogni brand nell'analisi nel seguente modo

In [ ]:
penetrazione_brand = df['Brand'].value_counts()
penetrazione_brand / totale_campione

## Calcolo del CDI

Il CDI é calcolato il tasso di penetrazione della categoria merceologica su ognuna delle dimensioni in esame, indicizzato a 100.

Risponde alla domanda "Quante persone appartenenti a ognuno dei segmenti target consumano abitualmente tonno in scatola?"

Vediamo i segmenti unici presenti dentro il nostro dataset

In [ ]:

segmenti_target = df['Segmento'].unique().tolist()
segmenti_target

Vediamo il numero di persone appartenenti a ogni segmento

In [ ]:

rispondenti_per_segmento = df['Segmento'].value_counts()
rispondenti_per_segmento

Vediamoli in un grafico

In [ ]:
plt.hist(df['Segmento'])
plt.plot()

Ora dediamo il numero di persone appartenenti a ogni segmento __che acquista tonno in scatola__

In [ ]:
consumatori_per_segmento = df[df['Brand'].notnull()]['Segmento'].value_counts()
consumatori_per_segmento

Salviamo in una variabile la diffusione del tonno in scatola in ogni segmento

In [ ]:
diffusione_per_segmento = pd.Series()
for seg in segmenti_target:
    diffusione_per_segmento[seg] = consumatori_per_segmento[seg] / rispondenti_per_segmento[seg]

diffusione_per_segmento

Per indicizzare un indicatore a 100 si divide ogni numero per la media sulla popolazione (o campione)

In [ ]:
cdi = 100.0 * diffusione_per_segmento / penetrazione_categoria
cdi

# Calcolo del BDI

Il Brand Development Index indica quanto ogni marca sia diffusa in una determinata dimensione di analisi (un segmento nel nostro caso) rispetto alla penetrazione del brand nell'intera popolazione. Cominciamo salvando la diffusione del brand nella popolazione con la seguente istruzione:

In [ ]:
diffusione_brand_popolazione = df['Brand'].value_counts()
diffusione_brand_popolazione = diffusione_brand_popolazione / len(df)
diffusione_brand_popolazione

il seguente codice itera su tutte le permutazioni di brand/segmento, calcolando il BDI (che é la diffusione di un brand in un determinato segmento, indicizzata sulla diffusione del brand nell'intero campione)

In [ ]:
# Create a DataFrame for the results
results_df = pd.DataFrame(columns=['brand', 'segmento', 'consumatori_per_brand_e_segmento', 'consumatori_per_segmento', 'diffusione_brand_nel_segmento', 'diffusione_brand_popolazione', 'BDI', 'CDI'])

rows_list = []

for brand in target_brands:
    for segmento in segmenti_target:
        # qui prendiamo il numero di consumatori del nostro segmento che acquistano regolarmente il brand
        consumatori_per_brand_e_segmento = len(df[(df['Brand'] == brand) & (df['Segmento'] == segmento)])
        # il numero di consumatori totali di questo segmento é giá in "rispondenti_per_segmento", da qui possiamo sapere la diffusione del brand nel segmento
        diffusione_brand_nel_segmento = consumatori_per_brand_e_segmento / rispondenti_per_segmento[segmento]
        # la diffusione del brand nell'intera popolazione é in diffusione_brand_popolazione, quindi possiamo calcolare il bdi

        bdi = 100.0 * diffusione_brand_nel_segmento / diffusione_brand_popolazione[brand]

        row = {
            'brand': brand,
            'segmento': segmento,
            'consumatori_per_brand_e_segmento': consumatori_per_brand_e_segmento,
            'consumatori_per_segmento': consumatori_per_segmento[segmento],
            'diffusione_brand_nel_segmento': diffusione_brand_nel_segmento,
            'diffusione_brand_popolazione': diffusione_brand_popolazione[brand],
            'BDI': bdi,
            'CDI': cdi[segmento],
        }
        rows_list.append(row)

results_df = pd.concat([results_df, pd.DataFrame(rows_list)], ignore_index=True)
results_df

In [ ]:
results_df['BDI_CDI'] = results_df.BDI / results_df.CDI

Come ultima cosa, per esportare in excel, "pivottiamo" mantenendo sulle righe i segmenti target e sulle colonne i brand

In [ ]:
pivoted_df = results_df.pivot_table(index='segmento', 
                                   columns='brand', 
                                   values=['BDI', 'CDI', 'BDI_CDI'], 
                                   aggfunc='first')

pivoted_df

In [ ]:
pivoted_df.to_excel('pivoted_df.xlsx')